In [2]:
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np

## set the working directory

In [3]:
# check current directory
os.getcwd()

'/Users/jessesnaust/Documents/python scripts'

In [4]:
# change directory
os.chdir('/Volumes/WD_1TB/')
os.getcwd()

'/Volumes/WD_1TB'

In [57]:
# change back
os.chdir('/Users/jessesnaust/Documents/python scripts')
os.getcwd()

'/Users/jessesnaust/Documents/python scripts'

## set up functions

In [5]:
def variance_of_laplacian(image):
# compute the Laplacian of the image and then return the focus
# measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()

In [6]:
# find over-exposed areas
# returns the list of blobs, mask

from skimage import measure

def find_blobs(input_image):
    # Gaussian blur
    blr = cv2.GaussianBlur(input_image,(5,5),cv2.BORDER_DEFAULT)

    # threshold
    th = cv2.threshold(blr, 235, 255, cv2.THRESH_BINARY)[1] # cv2.threshold gives 2 outputs, ret & th
                                                            # select the 2nd one
    # perform erosions and dilations to remove blobs of noise from image
    th = cv2.erode(th, None, iterations=2)
    th = cv2.dilate(th, None, iterations=4)
   
    # perform connected component analysis on thresholded image
    labels = measure.label(th, neighbors=8, background=0)
    
    # initialize a mask to store only the large components
    mask = np.zeros(th.shape, dtype='uint8')
    
    # initialize blobList to store a list of over-exposed cells found
    #blobList = [0]
    # loop over unique components
    for label in np.unique(labels):
        #ignore the background label
        if label ==0: #ignore background label
            continue
            
        # construct label mask for current label
        labelMask = np.zeros(th.shape, dtype='uint8')
        labelMask[labels == label] = 255
        
        # count the # of non-zero pixels 
        numPixels = cv2.countNonZero(labelMask)
    
        # if # of pixels in the component is greater than a threshold,
        # then add it to the overall mask
        if numPixels > 1000:
            mask = cv2.add(mask, labelMask)
            #blobList.append(label)
    return mask

In [7]:
# function to blur out blobs
def blur_blobs(input_image, input_mask):
    blr = cv2.GaussianBlur(input_image, (101, 101), 0) # blur image
    dilatedMask = cv2.dilate(mask, None, iterations=25)
    output_image = np.where(dilatedMask, blr, input_image) # mask over original image with blurred
    return output_image

In [8]:
def create_folder(file_path, folder_name):
    if not os.path.exists(file_path + folder_name):
        print("Creating folder '{}'.".format(folder_name))
        os.makedirs(file_path + folder_name)
    else:
        print("Folder '{}' already exists.".format(folder_name))

In [9]:
# get unique file names
def get_names(folder_path):
    ID_list = []
    for entry in os.scandir(folder_path):
        if entry.name.endswith('d0.TIF'):
            fName = entry.name
            index = fName.find('_')
            matchID = fName[index+1:-6]
            ID_list.append(matchID)
    return ID_list

In [41]:
# auto-adjust histogram
def hist_adj(img, clahe, offset):
    hist, bins = np.histogram(img.flatten(),256,[0,256])
    cdf = hist.cumsum()
    #  get the cumulative distribution function
    cdf_normalized = cdf * hist.max()/ cdf.max()
    # find the slope of the cdf
    slope = np.diff(cdf_normalized)/np.diff(bins[:-1])
    # find the index of the first non-zero value of the derivative of the slope
    idx = np.nonzero(np.diff(slope))[0]
    # clahe
    if clahe == True:
        clahe = cv2.createCLAHE(clipLimit=0.5, tileGridSize=(5,5)) # set clipLimit to adj how much detail to bring up
        cl1 = clahe.apply(img)
        adjImg = cl1 + offset - idx[0]
    elif clahe == False:
        # slide the histogram over
        adjImg = img + offset - idx[0]
    return adjImg

In [32]:
def gamma_adj(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [50]:
def illum_corr(image):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (100,100))
    topHat = cv2.morphologyEx(image, cv2.MORPH_TOPHAT, kernel)
    return topHat

## get to work

In [47]:
folder_path = "Cellomics_images/102919-M134I/"
create_folder(folder_path, 'not_blurry_merged')
#create_folder(file_path, 'blurry')

Folder 'not_blurry_merged' already exists.


In [48]:
dst_folder = folder_path+'not_blurry_merged/'

In [56]:
ID_list = get_names(folder_path)

for x in ID_list:
    print('ID = {}'.format(x))
    for entry in os.scandir(folder_path):
        if x in entry.name and entry.name.endswith('d0.TIF'):
            blue = cv2.imread(entry.path, 0)
            print(entry.name)
        if x in entry.name and entry.name.endswith('d1.TIF'):
            green = cv2.imread(entry.path, 0)
            print(entry.name)
        if x in entry.name and entry.name.endswith('d2.TIF'):
            red = cv2.imread(entry.path, 0)
            print(entry.name)
        else:
            continue
        
        # use DAPI to do 1st rnd of filtering
        edgesBlue = cv2.Canny(blue, 100, 200)
        fmBlue = variance_of_laplacian(edgesBlue)
        
        if fmBlue > 1400:
            print('fm = {:2f}'.format(fmBlue))
            # blur out over-exposed cells
            mask = find_blobs(red)
            blrred = blur_blobs(red, mask)
                
            # remove air bubbles
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))  # taking a matrix of size 5 as kernel
            morph = cv2.morphologyEx(blrred, cv2.MORPH_OPEN, kernel) # (erosion followed by dilation)
            
            # 2nd rnd of edge detect, after processing tricky images
            edgesRed = cv2.Canny(morph, 50, 200)
            fmRed = variance_of_laplacian(edgesRed)
            
            if fmRed > 230:
                # illumination correction on all channels
                blue = illum_corr(blue)
                green = hist_adj(green, clahe=False, offset=0)
                red = hist_adj(red, clahe=False, offset=0)
                
                mergedImg = cv2.merge([blue, green, red])
                fName = 'merged_' + str(x) + '.TIF'
                print('Saving file - {}...'.format(fName))
                cv2.imwrite(dst_folder + fName, mergedImg)

ID = 191029100001_B03f00
MFGTMP_191029100001_B03f00d0.TIF
MFGTMP_191029100001_B03f00d1.TIF
MFGTMP_191029100001_B03f00d2.TIF
fm = 1750.493717
Saving file merged_191029100001_B03f00.TIF...
ID = 191029100001_B03f01
MFGTMP_191029100001_B03f01d0.TIF
MFGTMP_191029100001_B03f01d1.TIF
MFGTMP_191029100001_B03f01d2.TIF
fm = 1536.425971
ID = 191029100001_B03f02
MFGTMP_191029100001_B03f02d0.TIF
MFGTMP_191029100001_B03f02d1.TIF
MFGTMP_191029100001_B03f02d2.TIF
ID = 191029100001_B03f03
MFGTMP_191029100001_B03f03d0.TIF
MFGTMP_191029100001_B03f03d1.TIF
MFGTMP_191029100001_B03f03d2.TIF
ID = 191029100001_B03f04
MFGTMP_191029100001_B03f04d0.TIF
MFGTMP_191029100001_B03f04d1.TIF
MFGTMP_191029100001_B03f04d2.TIF
fm = 1421.702528
ID = 191029100001_B03f05
MFGTMP_191029100001_B03f05d0.TIF
MFGTMP_191029100001_B03f05d1.TIF
MFGTMP_191029100001_B03f05d2.TIF
fm = 1445.825456
Saving file merged_191029100001_B03f05.TIF...
ID = 191029100001_B03f06
MFGTMP_191029100001_B03f06d0.TIF
MFGTMP_191029100001_B03f06d1.TIF
MFGTM

ID = 191029120001_B03f144
MFGTMP_191029120001_B03f144d0.TIF
MFGTMP_191029120001_B03f144d1.TIF
MFGTMP_191029120001_B03f144d2.TIF
fm = 4844.119857
Saving file merged_191029120001_B03f144.TIF...
ID = 191029120001_B03f145
MFGTMP_191029120001_B03f145d0.TIF
MFGTMP_191029120001_B03f145d1.TIF
MFGTMP_191029120001_B03f145d2.TIF
fm = 8878.540299
Saving file merged_191029120001_B03f145.TIF...
ID = 191029120001_B03f146
MFGTMP_191029120001_B03f146d0.TIF
MFGTMP_191029120001_B03f146d1.TIF
MFGTMP_191029120001_B03f146d2.TIF
ID = 191029120001_B03f147
MFGTMP_191029120001_B03f147d0.TIF
MFGTMP_191029120001_B03f147d1.TIF
MFGTMP_191029120001_B03f147d2.TIF
fm = 2099.625063
Saving file merged_191029120001_B03f147.TIF...
ID = 191029120001_B03f148
MFGTMP_191029120001_B03f148d0.TIF
MFGTMP_191029120001_B03f148d1.TIF
MFGTMP_191029120001_B03f148d2.TIF
fm = 2443.051234
Saving file merged_191029120001_B03f148.TIF...
ID = 191029120001_B03f149
MFGTMP_191029120001_B03f149d0.TIF
MFGTMP_191029120001_B03f149d1.TIF
MFGTMP_191

Saving file merged_191029120001_B03f196.TIF...
ID = 191029120001_B03f197
MFGTMP_191029120001_B03f197d0.TIF
MFGTMP_191029120001_B03f197d1.TIF
MFGTMP_191029120001_B03f197d2.TIF
fm = 2117.422687
Saving file merged_191029120001_B03f197.TIF...
ID = 191029120001_B03f198
MFGTMP_191029120001_B03f198d0.TIF
MFGTMP_191029120001_B03f198d1.TIF
MFGTMP_191029120001_B03f198d2.TIF
fm = 4220.644488
ID = 191029120001_B03f199
MFGTMP_191029120001_B03f199d0.TIF
MFGTMP_191029120001_B03f199d1.TIF
MFGTMP_191029120001_B03f199d2.TIF
fm = 3281.648575
Saving file merged_191029120001_B03f199.TIF...
ID = 191029120001_B03f200
MFGTMP_191029120001_B03f200d0.TIF
MFGTMP_191029120001_B03f200d1.TIF
MFGTMP_191029120001_B03f200d2.TIF
fm = 5769.597025
Saving file merged_191029120001_B03f200.TIF...
ID = 191029120001_B03f201
MFGTMP_191029120001_B03f201d0.TIF
MFGTMP_191029120001_B03f201d1.TIF
MFGTMP_191029120001_B03f201d2.TIF
fm = 2035.876030
Saving file merged_191029120001_B03f201.TIF...
ID = 191029120001_B03f202
MFGTMP_1910291

Saving file merged_191029120001_B03f225.TIF...
ID = 191029120001_B03f226
MFGTMP_191029120001_B03f226d0.TIF
MFGTMP_191029120001_B03f226d1.TIF
MFGTMP_191029120001_B03f226d2.TIF
fm = 1581.137102
Saving file merged_191029120001_B03f226.TIF...
ID = 191029120001_B03f227
MFGTMP_191029120001_B03f227d0.TIF
MFGTMP_191029120001_B03f227d1.TIF
MFGTMP_191029120001_B03f227d2.TIF
ID = 191029120001_B03f228
MFGTMP_191029120001_B03f228d0.TIF
MFGTMP_191029120001_B03f228d1.TIF
MFGTMP_191029120001_B03f228d2.TIF
ID = 191029120001_B03f229
MFGTMP_191029120001_B03f229d0.TIF
MFGTMP_191029120001_B03f229d1.TIF
MFGTMP_191029120001_B03f229d2.TIF
ID = 191029120001_B03f22
MFGTMP_191029120001_B03f220d0.TIF
MFGTMP_191029120001_B03f220d1.TIF
MFGTMP_191029120001_B03f220d2.TIF
fm = 3329.026284
Saving file merged_191029120001_B03f22.TIF...
MFGTMP_191029120001_B03f221d0.TIF
MFGTMP_191029120001_B03f221d1.TIF
MFGTMP_191029120001_B03f221d2.TIF
fm = 4637.183615
MFGTMP_191029120001_B03f222d0.TIF
MFGTMP_191029120001_B03f222d1.TIF


MFGTMP_191029120001_B03f254d2.TIF
fm = 1648.668905
ID = 191029120001_B03f255
MFGTMP_191029120001_B03f255d0.TIF
MFGTMP_191029120001_B03f255d1.TIF
MFGTMP_191029120001_B03f255d2.TIF
fm = 6152.153153
Saving file merged_191029120001_B03f255.TIF...
ID = 191029120001_B03f256
MFGTMP_191029120001_B03f256d0.TIF
MFGTMP_191029120001_B03f256d1.TIF
MFGTMP_191029120001_B03f256d2.TIF
fm = 4006.700777
Saving file merged_191029120001_B03f256.TIF...
ID = 191029120001_B03f257
MFGTMP_191029120001_B03f257d0.TIF
MFGTMP_191029120001_B03f257d1.TIF
MFGTMP_191029120001_B03f257d2.TIF
fm = 7891.050540
ID = 191029120001_B03f258
MFGTMP_191029120001_B03f258d0.TIF
MFGTMP_191029120001_B03f258d1.TIF
MFGTMP_191029120001_B03f258d2.TIF
fm = 5698.096378
ID = 191029120001_B03f259
MFGTMP_191029120001_B03f259d0.TIF
MFGTMP_191029120001_B03f259d1.TIF
MFGTMP_191029120001_B03f259d2.TIF
fm = 7193.656029
ID = 191029120001_B03f25
MFGTMP_191029120001_B03f250d0.TIF
MFGTMP_191029120001_B03f250d1.TIF
MFGTMP_191029120001_B03f250d2.TIF
MFG

Saving file merged_191029120001_B03f27.TIF...
ID = 191029120001_B03f280
MFGTMP_191029120001_B03f280d0.TIF
MFGTMP_191029120001_B03f280d1.TIF
MFGTMP_191029120001_B03f280d2.TIF
fm = 2035.938027
Saving file merged_191029120001_B03f280.TIF...
ID = 191029120001_B03f281
MFGTMP_191029120001_B03f281d0.TIF
MFGTMP_191029120001_B03f281d1.TIF
MFGTMP_191029120001_B03f281d2.TIF
ID = 191029120001_B03f282
MFGTMP_191029120001_B03f282d0.TIF
MFGTMP_191029120001_B03f282d1.TIF
MFGTMP_191029120001_B03f282d2.TIF
fm = 2363.302946
Saving file merged_191029120001_B03f282.TIF...
ID = 191029120001_B03f283
MFGTMP_191029120001_B03f283d0.TIF
MFGTMP_191029120001_B03f283d1.TIF
MFGTMP_191029120001_B03f283d2.TIF
fm = 4731.008771
ID = 191029120001_B03f284
MFGTMP_191029120001_B03f284d0.TIF
MFGTMP_191029120001_B03f284d1.TIF
MFGTMP_191029120001_B03f284d2.TIF
fm = 4209.606241
Saving file merged_191029120001_B03f284.TIF...
ID = 191029120001_B03f285
MFGTMP_191029120001_B03f285d0.TIF
MFGTMP_191029120001_B03f285d1.TIF
MFGTMP_1910

MFGTMP_191029120001_B03f310d1.TIF
MFGTMP_191029120001_B03f310d2.TIF
ID = 191029120001_B03f311
MFGTMP_191029120001_B03f311d0.TIF
MFGTMP_191029120001_B03f311d1.TIF
MFGTMP_191029120001_B03f311d2.TIF
ID = 191029120001_B03f312
MFGTMP_191029120001_B03f312d0.TIF
MFGTMP_191029120001_B03f312d1.TIF
MFGTMP_191029120001_B03f312d2.TIF
ID = 191029120001_B03f313
MFGTMP_191029120001_B03f313d0.TIF
MFGTMP_191029120001_B03f313d1.TIF
MFGTMP_191029120001_B03f313d2.TIF
fm = 2766.261292
ID = 191029120001_B03f314
MFGTMP_191029120001_B03f314d0.TIF
MFGTMP_191029120001_B03f314d1.TIF
MFGTMP_191029120001_B03f314d2.TIF
fm = 3044.388126
ID = 191029120001_B03f315
MFGTMP_191029120001_B03f315d0.TIF
MFGTMP_191029120001_B03f315d1.TIF
MFGTMP_191029120001_B03f315d2.TIF
fm = 2376.511602
Saving file merged_191029120001_B03f315.TIF...
ID = 191029120001_B03f316
MFGTMP_191029120001_B03f316d0.TIF
MFGTMP_191029120001_B03f316d1.TIF
MFGTMP_191029120001_B03f316d2.TIF
fm = 1933.431100
Saving file merged_191029120001_B03f316.TIF...
ID

Saving file merged_191029120001_B03f66.TIF...
ID = 191029120001_B03f67
MFGTMP_191029120001_B03f67d0.TIF
MFGTMP_191029120001_B03f67d1.TIF
MFGTMP_191029120001_B03f67d2.TIF
fm = 1868.565842
Saving file merged_191029120001_B03f67.TIF...
ID = 191029120001_B03f68
MFGTMP_191029120001_B03f68d0.TIF
MFGTMP_191029120001_B03f68d1.TIF
MFGTMP_191029120001_B03f68d2.TIF
fm = 5475.470922
Saving file merged_191029120001_B03f68.TIF...
ID = 191029120001_B03f69
MFGTMP_191029120001_B03f69d0.TIF
MFGTMP_191029120001_B03f69d1.TIF
MFGTMP_191029120001_B03f69d2.TIF
ID = 191029120001_B03f70
MFGTMP_191029120001_B03f70d0.TIF
MFGTMP_191029120001_B03f70d1.TIF
MFGTMP_191029120001_B03f70d2.TIF
fm = 3437.610483
Saving file merged_191029120001_B03f70.TIF...
ID = 191029120001_B03f71
MFGTMP_191029120001_B03f71d0.TIF
MFGTMP_191029120001_B03f71d1.TIF
MFGTMP_191029120001_B03f71d2.TIF
ID = 191029120001_B03f72
MFGTMP_191029120001_B03f72d0.TIF
MFGTMP_191029120001_B03f72d1.TIF
MFGTMP_191029120001_B03f72d2.TIF
fm = 3064.294195
Savi

In [ ]:
# now do everything
#for entry in os.scandir(folder_path):
    if entry.name.endswith('d2.TIF'):
        image = cv2.imread(entry.path,0)
        print(entry.name)
        if np.median(image) < 80: # remove noisy images
            # 1st round of edge detection, to remove obviously blurry images
            edges = cv2.Canny(image, 100, 200) # Canny edge detection
            fm = variance_of_laplacian(edges)# calculate focus measure (fm)
            
        # process only not blurry images
            if fm > 300:
                # blur out over-exposed cells
                mask = find_blobs(image)
                blrred = blur_blobs(image, mask)
                
                # remove air bubbles
                kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))  # taking a matrix of size 5 as kernel
                morph = cv2.morphologyEx(blrred, cv2.MORPH_OPEN, kernel) # (erosion followed by dilation)
            
                # 2nd rnd of edge detect, after processing tricky images
                edges2 = cv2.Canny(morph, 50, 200)
                fm2 = variance_of_laplacian(edges2)
                value = "{:.2f}".format(fm2)
                file_name = value + "_" + entry.name
                if fm2 > 230:
                    cv2.imwrite(file_path + "/not_blurry/" + file_name, image )
                else:
                    cv2.imwrite(file_path + "/blurry/" + file_name, image)

In [30]:
ID_list = get_names(folder_path)

for x in ID_list:
    print(x)
    for entry in os.scandir(folder_path):
        if x in entry.name and entry.name.endswith('d0.TIF'):
             print(x, 'blue')
        if x in entry.name and entry.name.endswith('d1.TIF'):
             print(x, 'green')
        if x in entry.name and entry.name.endswith('d2.TIF'):
             print(x, 'red')

B01f00
B01f00 blue
B01f00 green
B01f00 red
B01f01
B01f01 blue
B01f01 green
B01f01 red
B01f02
B01f02 blue
B01f02 green
B01f02 red
B01f03
B01f03 blue
B01f03 green
B01f03 red
B01f04
B01f04 blue
B01f04 green
B01f04 red
B01f05
B01f05 blue
B01f05 green
B01f05 red
B01f06
B01f06 blue
B01f06 green
B01f06 red
B01f07
B01f07 blue
B01f07 green
B01f07 red
B01f08
B01f08 blue
B01f08 green
B01f08 red
B01f09
B01f09 blue
B01f09 green
B01f09 red
B01f100
B01f100 blue
B01f100 green
B01f100 red
B01f101
B01f101 blue
B01f101 green
B01f101 red
B01f102
B01f102 blue
B01f102 green
B01f102 red
B01f103
B01f103 blue
B01f103 green
B01f103 red
B01f104
B01f104 blue
B01f104 green
B01f104 red
B01f105
B01f105 blue
B01f105 green
B01f105 red
B01f106
B01f106 blue
B01f106 green
B01f106 red
B01f107
B01f107 blue
B01f107 green
B01f107 red
B01f108
B01f108 blue
B01f108 green
B01f108 red
B01f109
B01f109 blue
B01f109 green
B01f109 red
B01f110
B01f110 blue
B01f110 green
B01f110 red
B01f111
B01f111 blue
B01f111 green
B01f111 red
B01f11

B01f262
B01f262 blue
B01f262 green
B01f262 red
B01f263
B01f263 blue
B01f263 green
B01f263 red
B01f264
B01f264 blue
B01f264 green
B01f264 red
B01f265
B01f265 blue
B01f265 green
B01f265 red
B01f266
B01f266 blue
B01f266 green
B01f266 red
B01f267
B01f267 blue
B01f267 green
B01f267 red
B01f268
B01f268 blue
B01f268 green
B01f268 red
B01f269
B01f269 blue
B01f269 green
B01f269 red
B01f26
B01f26 blue
B01f26 green
B01f26 red
B01f26 blue
B01f26 green
B01f26 red
B01f26 blue
B01f26 green
B01f26 red
B01f26 blue
B01f26 green
B01f26 red
B01f26 blue
B01f26 green
B01f26 red
B01f26 blue
B01f26 green
B01f26 red
B01f26 blue
B01f26 green
B01f26 red
B01f26 blue
B01f26 green
B01f26 red
B01f26 blue
B01f26 green
B01f26 red
B01f26 blue
B01f26 green
B01f26 red
B01f26 blue
B01f26 green
B01f26 red
B01f270
B01f270 blue
B01f270 green
B01f270 red
B01f271
B01f271 blue
B01f271 green
B01f271 red
B01f272
B01f272 blue
B01f272 green
B01f272 red
B01f273
B01f273 blue
B01f273 green
B01f273 red
B01f274
B01f274 blue
B01f274 gree

In [ ]:
elif fm > 300:
            max_cnt_len = find_maxContours(edges) # find the max contour length
            if max_cnt_len > 200: # if there're air bubbles in the image
                cv2.imwrite(file_path + "not_blurry/" + "bubble_" + value + entry.name, image )
            elif len(blobList) > 0: # if blobs are found, label the image file w/ "OE" prefix
                cv2.imwrite(file_path + "not_blurry/" + "OE_" + value + "_" + entry.name, image )
            else:
                cv2.imwrite(file_path + "not_blurry/" + value + "_" + entry.name, image )

In [ ]:
           if len(blobList) > 1:
                blobList, mask = find_blobs(image) # find over-exposed cells
                blrBlob = blur_blobs(image, mask) # blur out over-exposed cells